# Tips for cost-effective usage of Azure Cosmos DB at any scale

In this notebook, we'll learn how to scale Azure Cosmos containers manually and using autoscale.

To run this C# notebook, be sure to select the **CSharp** kernel in the command bar, so we can get all the language support features we need.


## Create database and containers
First, we'll create a new database and container to hold our data. Note when we create the container, we select a value for the partition key: we'll partition our data by the item id value, as it has a high cardinality (important for workloads during a lot of writes, e.g. IOT workloads) and evenly distributes the request and storage volume. Choosing a good partition key is "key" to getting good scale and performance from Azure Cosmos DB, so it's important we follow the [best practices](https://docs.microsoft.com/azure/cosmos-db/partitioning-overview)!

### To start, we'll create a container that has the minimum 400 RU/s of throughput.

In [1]:
using System;
using Microsoft.Azure.Cosmos;
using System.Collections;
using System.Collections.Generic;

// Initialize a new instance of CosmosClient using the built-in account endpoint and key parameters
CosmosClient cosmosClient = new CosmosClient(Cosmos.Endpoint, Cosmos.Key);

// Create a new database and a new container
Microsoft.Azure.Cosmos.Database database = await cosmosClient.CreateDatabaseIfNotExistsAsync("WebsiteData");

ContainerProperties containerProperties = new ContainerProperties()
{
    Id = "low-throughput",
    PartitionKeyPath = "/id",
    IndexingPolicy = new IndexingPolicy()
   {
        Automatic = false,
        IndexingMode = IndexingMode.None,
   }
};

Container container = await database.CreateContainerIfNotExistsAsync(containerProperties, throughput: 400);

Display.AsMarkdown(@"
Created database WebsiteData and container Sales. You can see these new resources by refreshing your resource pane under the Data section.
");


Created database WebsiteData and container Sales. You can see these new resources by refreshing your resource pane under the Data section.


### Ingesting data

We'll ingest an example retail dataset with a similar structure to the retail dataset from the previous example. This retail dataset has approx. 2,600 documents that are 1 KB each. We can estimate how long it will take to ingest 2,600 with our 400 RU/s of provisioned throughput.

Since each document is 1 KB, it will cost about 5 RUs of throughput to ingest each document. Since we have 400 RU/s of provisioned throughput, we have 400 RUs of throughput available each second. Therefore, we can ingest approximately 80 documents per second.

With a ingestion rate of 80 documents per second, it will take approx. 30 seconds to ingest our full dataset of 2,600 documents.

In [2]:
%%upload --databaseName WebsiteData --containerName low-throughput --url https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/websiteData.json

Documents successfully uploaded to low-throughput 
Total number of documents imported: 
  Success: 2654 
  Failure: 0 
Total time taken : 00:00:58 hours 
Total RUs consumed : 26274.600000001028

### Scaling up throughput

For workloads with small documents, 400 RU/s allows for 80 writes/sec. While this is sufficient scale in many cases, some customers need a higher provisioned throughput. In the next code sample, we'll create a new container with 4,000 RU/s, allowing data ingestion at 10x the rate of 400 RU/s.

When scaling Azure Cosmos containers, it's good to keep in mind that:

1. For most changes that are less than a 10x increase, containers scale instantly.
2. Scaling a container (either up or down) will never cause downtime. 
3. You can scale a container programmatically (uses SDKs or Azure CLI) as well as in the Azure Portal

Let's try it out!

In [3]:
// Create a new container with 4,000 RUs
ContainerProperties containerProperties = new ContainerProperties()
{
    Id = "high-throughput",
    PartitionKeyPath = "/id",
    IndexingPolicy = new IndexingPolicy()
   {
        Automatic = false,
        IndexingMode = IndexingMode.None,
   }
};

Container container = await database.CreateContainerIfNotExistsAsync(containerProperties, throughput: 4000);

Display.AsMarkdown(@"
Created high-throughput container with 4,000 RU/s
");


Created high-throughput container with 4,000 RU/s


In [ ]:
%%upload --databaseName WebsiteData --containerName high-throughput --url https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/websiteData.json

### Interpreting results

In this case, by scaling up our scale-test container, we were able to increase the ingestion rate by 10x. In Azure Cosmos DB, you can scale throughput manually or automatically, using autoscale. For most customers, it's impractical to constantly monitor provisioned throughput. By configuring autoscale, you can set a maximum throughput value and Azure Cosmos DB will automatically scale your throughput, scaling between 10% of the max throughput to the full max throughput.

Let's try out autoscale! We'll repeat the earlier test but this time set a max autoscale throughput of 4,000 RU/s. That way, we'll be able to use the full 4,000 RUs of throughput when required, but have the advantage of automatically scaling down to 400 RU/s when the added throughput isn't required.

In [ ]:
using System;
using Microsoft.Azure.Cosmos;
using System.Collections;
using System.Collections.Generic;

// Initialize a new instance of CosmosClient using the built-in account endpoint and key parameters
CosmosClient cosmosClient = new CosmosClient(Cosmos.Endpoint, Cosmos.Key);

// Create a new database and a new container
Microsoft.Azure.Cosmos.Database database = await cosmosClient.CreateDatabaseIfNotExistsAsync("WebsiteData");

ContainerProperties containerProperties = new ContainerProperties()
{
    Id = "autoscale-test",
    PartitionKeyPath = "/id",
    IndexingPolicy = new IndexingPolicy()
   {
        Automatic = false,
        IndexingMode = IndexingMode.None,
   }
};

Container container = await database.CreateContainerIfNotExistsAsync(containerProperties, ThroughputProperties.CreateAutoscaleThroughput(4000));

Display.AsMarkdown(@"
Created database WebsiteData and container autoscale-test. You can see these new resources by refreshing your resource pane under the Data section.
");

In [ ]:
%%upload --databaseName WebsiteData --containerName autoscale-test --url https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/websiteData.json

## Learn more about options for scaling throughput in Azure Cosmos DB

https://www.youtube.com/watch?v=CgYQo6uHyt0&t=1s